# Proyecto - Asistente de IA para aerolíneas

Ahora, reuniremos lo que hemos aprendido para crear un asistente de IA para atención al cliente para una aerolínea.

In [12]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [13]:
# Inicialización

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key Sin Configurar")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [14]:
system_message = "Eres un asistente útil para una aerolínea llamada FlightAI. "
system_message += "Da respuestas breves y corteses, de no más de una oración. "
system_message += "Se siempre preciso. Si no sabes la respuesta, dilo."

In [15]:
# Esta función parece bastante más simple que la de mi video, porque estamos aprovechando las últimas actualizaciones de Gradio.

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


## Herramientas

Las herramientas son una característica increíblemente poderosa que ofrecen los LLM de vanguardia.

Con las herramientas, puedes escribir una función y hacer que el LLM llame a esa función como parte de su respuesta.

Suena casi espeluznante... ¿le estamos dando el poder de ejecutar código en nuestra máquina?

Bueno, más o menos.

In [45]:
# Vamos a hacer una función útil

ticket_prices = {"londres": "799€", "parís": "899€", "tokyo": "1400€", "berlín": "499€", "medellín": "700€"}
ticket_times = {"londres": "07:00", "parís": "11:00", "tokyo": "22:00", "berlín": "14:55"}

def get_ticket_price(destination_city):
    print(f"Se solicitó la herramienta get_ticket_price para {destination_city}")
    city = destination_city.lower()
    price = ticket_prices.get(city, "Unknown")
    print(f"Se solicitó la herramienta get_ticket_price para {city} precio: {price}")
    return price

def get_ticket_time(destination_city):
    print(f"Se solicitó la herramienta get_ticket_times para {destination_city}")
    city = destination_city.lower()
    return ticket_times.get(city, "Unknown")

In [46]:
get_ticket_price("Berlin")
get_ticket_time("parís")
get_ticket_time("Medellín")

Se solicitó la herramienta get_ticket_price para Berlin
Se solicitó la herramienta get_ticket_price para berlin precio: Unknown
Se solicitó la herramienta get_ticket_times para parís
Se solicitó la herramienta get_ticket_times para Medellín


'Unknown'

In [47]:
# Hay una estructura de diccionario particular que se requiere para describir nuestra función:

price_function = {
    "name": "get_ticket_price",
    "description": "Obtén el precio de un billete de ida y vuelta a la ciudad de destino. Llámalo siempre que necesites saber el precio del billete, por ejemplo, cuando un cliente pregunte '¿Cuánto cuesta un billete a esta ciudad?'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "La ciudad a la que se viaja",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

time_function = {
    "name": "get_ticket_time",
    "description": "Obtén la hora de salida de un vuelo para una ciudad en especifico. Llámalo siempre que necesites saber el horario de salida, por ejemplo, cuando un cliente pregunte '¿a que horas sale el vuelo x, o A que hora sale el vuelo'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "La ciudad a la que el cliente desea viajar",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [48]:
# Y esto está incluido en una lista de herramientas:

tools = [{"type": "function", "function": price_function},
        {"type": "function", "function": time_function}]

## Cómo hacer que OpenAI use nuestra herramienta

Hay algunos detalles complicados para permitir que OpenAI "llame a nuestra herramienta"

Lo que realmente hacemos es darle al LLM la oportunidad de informarnos que quiere que ejecutemos la herramienta.

Así es como se ve la nueva función de chat:

In [49]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        tool_responses = handle_tool_call(message)
        messages.append(message)
        messages.extend(tool_responses)  # añadir todos
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [50]:
#Tenemos que escribir esa función handle_tool_call:

def handle_tool_call(message):
    responses = []
    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get("destination_city")

        if tool_call.function.name == "get_ticket_price":
            result = get_ticket_price(city)
            responses.append({
                "role": "tool",
                "content": json.dumps({"destination_city": city, "price": result}),
                "tool_call_id": tool_call.id
            })

        elif tool_call.function.name == "get_ticket_time":
            result = get_ticket_time(city)
            responses.append({
                "role": "tool",
                "content": json.dumps({"destination_city": city, "time": result}),
                "tool_call_id": tool_call.id
            })

    return responses

In [51]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7872
* To create a public link, set `share=True` in `launch()`.


Se solicitó la herramienta get_ticket_price para Medellín
Se solicitó la herramienta get_ticket_price para medellín precio: 700€
Se solicitó la herramienta get_ticket_price para Medellín
Se solicitó la herramienta get_ticket_price para medellín precio: 700€
Se solicitó la herramienta get_ticket_times para Medellín
